In [1]:
import math
import pandas as pd
import numpy as np

class LowpHVirusInactivation(object):
    
    def __init__(self,MassIn,VolumeIn,SUHoldBags):
        self.MassIn = MassIn
        self.VolumeIn = VolumeIn
        self.SUHoldBags = SUHoldBags
        
        
    def Sizing(self):
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        AcidicBufferVolume=self.VolumeIn*Protocol.get_value('Acid Buffer','Low pH Virus Inactivation')*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
        NeutBufferVolume=self.VolumeIn*(1+Protocol.get_value('Acid Buffer','Low pH Virus Inactivation'))*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*Protocol.get_value('Neutralisation Buffer','Low pH Virus Inactivation')
        VITankWV=self.VolumeIn*(1+Protocol.get_value('Acid Buffer','Low pH Virus Inactivation'))*(1+Protocol.get_value('Neutralisation Buffer','Low pH Virus Inactivation'))
        VolumeList=[AcidicBufferVolume,NeutBufferVolume,VITankWV]
        return AcidicBufferVolume,NeutBufferVolume,VITankWV,VolumeList
    
    def MassBalance(self):
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        StepYield=Protocol.get_value('Step Yield','Low pH Virus Inactivation')
        MassOut=self.MassIn*StepYield
        VolumeOut=self.VolumeIn*(1+Protocol.get_value('Acid Buffer','Low pH Virus Inactivation'))*(1+Protocol.get_value('Neutralisation Buffer','Low pH Virus Inactivation'))
        ConcentrationOut=MassOut*1000/VolumeOut
        return MassOut,VolumeOut,ConcentrationOut
    
    def Timeframe(self):
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #time to prepare for a batch
        PrepTime=Protocol.get_value('Preparation Time (hr)','Low pH Virus Inactivation')
        #time to operate a batch
        OperTime=Protocol.get_value('Operation Time (hr)','Low pH Virus Inactivation')
        #time to clean
        CIPTime=Protocol.get_value('CIP Time (hr)','Low pH Virus Inactivation')+Protocol.get_value('SIP Time (hr)','Low pH Virus Inactivation')
        #time to turnaround hold-tanks
        HoldTanksCIPandSIPTime=Protocol.get_value('CIP Time (hr)','Hold-Tanks')+Protocol.get_value('SIP Time (hr)','Hold-Tanks')
        return PrepTime,OperTime,CIPTime,HoldTanksCIPandSIPTime
    
    def EquipmentAndMaterials(self):
        #Load Equipment from Database
        Equipment=pd.read_excel('Database.xlsx',sheetname='Equipment',index_col=0)
        #Load Materials from Database
        Materials=pd.read_excel('Database.xlsx',sheetname='Materials',index_col=0)
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #acid and neutralisation buffer cost
        AcidCost=LowpHVirusInactivation.Sizing(self)[0]*Materials.get_value('Acid Buffer','Base Cost')
        NeutralisationCost=LowpHVirusInactivation.Sizing(self)[1]*Materials.get_value('Neutralisation Buffer','Base Cost')
        #Hold-tanks with GF, PW, WFI, CIP        
        HoldTankCostList=[]
        HoldTankGuardList=[]
        CIPBufferList=[]
        CIPBufferCostList=[]
        WFIVolumeList=[]
        PWVolumeList=[]
        WFIandPWCostList=[]
        for x in LowpHVirusInactivation.Sizing(self)[3]:
            if x/Protocol.get_value('Space Efficiency','Hold-Tanks')>Equipment.get_value('Hold-Tank','Max. Size'):
                NumberOfTanks=math.ceil((x/Protocol.get_value('Space Efficiency','Hold-Tanks'))/Equipment.get_value('Hold-Tank','Max. Size'))
            else:
                NumberOfTanks=1
            TankVolume=(x/Protocol.get_value('Space Efficiency','Hold-Tanks'))/NumberOfTanks
            if TankVolume<=10:
                CorrectedTankVolume=math.ceil(TankVolume)
            elif TankVolume<=100 and TankVolume>10:
                CorrectedTankVolume=int(math.ceil(TankVolume/10.0))*10
            elif TankVolume<=1000 and TankVolume>100:
                CorrectedTankVolume=int(math.ceil(TankVolume/100.0))*100
            else:
                CorrectedTankVolume=int(math.ceil(TankVolume/1000.0))*1000
            HoldTankCost=NumberOfTanks*Equipment.get_value('Hold-Tank','Base Cost')*(CorrectedTankVolume/Equipment.get_value('Hold-Tank','Base Size'))**Equipment.get_value('Hold-Tank','Scaling Factor')
            HoldTankGuard=NumberOfTanks*Materials.get_value('Hold-Tank Guard Filter','Base Cost')*(CorrectedTankVolume/Materials.get_value('Hold-Tank Guard Filter','Base Size'))**Materials.get_value('Hold-Tank Guard Filter','Scaling Factor')
            CIPBuffer=NumberOfTanks*(Protocol.get_value('Acid Buffer','Hold-Tanks')+Protocol.get_value('Caustic Buffer','Hold-Tanks'))*CorrectedTankVolume*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
            CIPBufferCost=CIPBuffer*Materials.get_value('CIP Buffer','Base Cost')
            HoldTankDiameter=NumberOfTanks*(CorrectedTankVolume*4000/(math.pi*Protocol.get_value('H/D tank ratio','Hold-Tanks')))**(1/3)
            WFIVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*HoldTankDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Hold-Tanks')/60*Protocol.get_value('WFI Rinse (min)','Hold-Tanks')
            PWVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*HoldTankDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Hold-Tanks')/60*Protocol.get_value('PW Rinse (min)','Hold-Tanks')
            WFIandPWCost=WFIVolume*Materials.get_value('WFI','Base Cost')+PWVolume*Materials.get_value('PW','Base Cost')
            HoldTankCostList.append(HoldTankCost)
            HoldTankGuardList.append(HoldTankGuard)
            CIPBufferList.append(CIPBuffer)
            CIPBufferCostList.append(CIPBufferCost)
            WFIVolumeList.append(WFIVolume)
            PWVolumeList.append(PWVolume)
            WFIandPWCostList.append(WFIandPWCost)
        TotalHoldTanksCost=sum(HoldTankCostList)
        TotalHoldGFCost=sum(HoldTankGuardList)
        TotalHoldCIPVolume=sum(CIPBufferList)
        TotalHoldCIPCost=sum(CIPBufferCostList)
        TotalHoldWFIVolume=sum(WFIVolumeList)
        TotalHoldPWVolume=sum(PWVolumeList)
        TotalHoldWFIandPWCost=sum(WFIandPWCostList)
        return TotalHoldTanksCost,AcidCost,NeutralisationCost,TotalHoldCIPCost,TotalHoldWFIandPWCost,TotalHoldGFCost
    
    def CostBreakdown(self):
        EquipmentPurchaseCost=sum(LowpHVirusInactivation.EquipmentAndMaterials(self)[0:1])
        #costs per batch
        ChemicalReagentsCost=sum(LowpHVirusInactivation.EquipmentAndMaterials(self)[1:5])
        ConsumablesCost=sum(LowpHVirusInactivation.EquipmentAndMaterials(self)[5:6])
        LabourCost=0
        return LabourCost,ChemicalReagentsCost,ConsumablesCost,EquipmentPurchaseCost

In [2]:
viexample=LowpHVirusInactivation(MassIn=10,VolumeIn=800,SUHoldBags=False)
print(viexample.Sizing())
print(viexample.MassBalance())
print(viexample.Timeframe())
print(viexample.EquipmentAndMaterials())
print(viexample.CostBreakdown())

(184.0, 220.80000000000001, 1152.0, [184.0, 220.80000000000001, 1152.0])
(9.5, 1152.0, 8.2465277777777786)
(0.5, 1.0, 6.0, 6.0)
(133866.94913206215, 239.20000000000002, 287.04000000000002, 8970.0, 666.84260490650695, 751.17088128045361)
(0, 10163.082604906507, 751.17088128045361, 133866.94913206215)
